In [1]:
import pandas as pd
import numpy as np
import time
import random
import os
import re

import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span

In [2]:
fn = '/home/snakeiiiiiii/Hamilton County House Project'
os.chdir(fn)
# open it...
homes =pd.read_excel('House Sales 2009 to 2023 Hamilton County.xlsx')

In [3]:
homes = homes.drop('Unnamed: 0',axis=1)
homes.address = homes.address.str.title()

In [4]:
# Initialize the spaCy model and Matcher
nlp = spacy.load("en_core_web_sm")

def is_alphanumeric(token):
    """Check if the token text is alphanumeric."""
    return re.match("^(?=.*[0-9])(?=.*[a-zA-Z])[a-zA-Z0-9]+$", token.text) is not None

def tag_address(address):
    """
    Tag the components of the address using the defined pattern.
    Returns a dictionary with the components tagged.
    """
    doc = nlp(address)
    tagged_components = {'st_num': None, 'apt_num': None, 'street': None}

    # Loop through the tokens to find matches based on conditions
    for i, token in enumerate(doc):
        if i == 0 and token.pos_ == "NUM":
            tagged_components['st_num'] = token.text
        elif i == 1 and is_alphanumeric(token):
            tagged_components['apt_num'] = token.text
        elif i == 1 and token.pos_=='NUM':
            tagged_components['apt_num'] = token.text
        elif i > 0:
            # Concatenate the remaining tokens as the street name
            tagged_components['street'] = " ".join([tok.text for tok in doc[i:]])
            break

    return tagged_components

# Apply the tag_address function to the first 10 addresses for demonstration
address_parts = []
for address in homes.address:
    address_parts.append(tag_address(address))

In [5]:
parts = pd.DataFrame.from_dict(address_parts)

In [6]:
homes = pd.concat([homes,parts],axis=1)

In [7]:
homes.loc[homes['apt_num'].isna(),'new_address'] = homes['st_num'] +' '+homes['street']
homes.loc[~homes['apt_num'].isna(),'new_address'] = homes['st_num'] +' '+homes['street']+' '+homes['apt_num']

In [8]:
final_df = homes.copy()

In [230]:
import googlemaps
from datetime import datetime

# Initialize the Google Maps client with your API key
gmaps = googlemaps.Client(key='sample')

def is_only_city_state_country_regex(address):
    # Regex pattern to match strings that might include a country
    pattern = r'^\D+, \D+(, \D+)?$'
    return re.match(pattern, address) is not None

def address_has_no_street_number(address):
    # Regex pattern to match street names without a leading street number
    # This regex assumes that the street number, if present, would typically appear at the start
    pattern = r'^(?!\d+\s)([\w\s]+\s(?:St|Street|Court|Ct|Cir|Circle|Cr|Ave|Avenue|Blvd|Boulevard|Rd|Road|Lane|Ln|Drive|Dr|Terrace|Terr))\b,?[\w\s,]*$'

    return re.match(pattern, address) is not None

def get_address_details_with_cities(address):
    # List of cities to try
    allowed_cities = ['Madeira, OH', 'Wyoming, OH', 'Loveland, OH', 'Mariemont, OH', 'Blue Ash, OH', 'Terrace Park, OH', 'Cincinnati, OH']
    for city in allowed_cities:
        # Construct the address with the current city from the list
        full_address_query = f"{address}, {city}"
        # Make the geocoding request
        response = gmaps.geocode(full_address_query)
        if response:
            # Get the formatted address from the response
            formatted_address = response[0]['formatted_address']
            # Check if the formatted address is not just city, state, country
            if not is_only_city_state_country_regex(formatted_address):
                if not address_has_no_street_number(formatted_address):
                    location = response[0]['geometry']['location']
                    if 'lat' in location and 'lng' in location:
                        # Return a dictionary with the required details
                        return {
                            'formatted_address': formatted_address,
                            'longitude': location['lng'],
                            'latitude': location['lat']
                        }    # If no valid address is found after all attempts
    print(full_address_query, "No valid address found.")
    return {'formatted_address': None, 'longitude': None, 'latitude': None}

In [238]:
results_df = final_df['new_address'].apply(get_address_details_with_cities).apply(pd.Series)

1260 Garden Cr, Cincinnati, OH No valid address found.
23 Murray Ave, Cincinnati, OH No valid address found.
5355 # D Cooper Rd, Cincinnati, OH No valid address found.


In [239]:
# Concatenate the new columns to the original DataFrame
final_df = pd.concat([final_df, results_df], axis=1)

In [242]:
final_df.loc[final_df.formatted_address.isna(),'new_address'] =['1260 Garden Cir, Cincinnati, OH 45215', '6309 Murray Rd, Cincinnati, OH 45227','5355 Cooper Rd, Blue Ash, OH 45242']

In [245]:
results_df = final_df[final_df.longitude.isna()]['new_address'].apply(get_address_details_with_cities).apply(pd.Series)

In [252]:
final_df.loc[final_df.latitude.isna(),'formatted_address'] = results_df['formatted_address']
final_df.loc[final_df.latitude.isna(),'longitude'] = results_df['longitude']
final_df.loc[final_df.latitude.isna(),'latitude'] = results_df['latitude']

In [254]:
final_df.loc[:, 'num_amount'] = (final_df['amount']
                                  .str.replace('$','')
                                  .str.replace(',','')
                                  .astype(int))

In [255]:
final_df['year']  = pd.to_datetime(final_df['transfer_date']).dt.year
final_df['month']  = pd.to_datetime(final_df['transfer_date']).dt.month

In [399]:
final_df.loc[final_df.address.str.contains('Grace'),'formatted_address'] = final_df[final_df.address.str.contains('Grace')]['formatted_address'].str.replace('45208','45277') 
final_df.loc[final_df.address.str.contains('Myrtle'),'formatted_address'] = final_df[final_df.address.str.contains('Myrtle')]['formatted_address'].str.replace('Loveland, OH 45140','Cincinnati, OH 45242') 
final_df.loc[final_df.address.str.contains('Cherry'),'formatted_address'] = final_df[final_df.address.str.contains('Cherry')]['formatted_address'].str.replace('Loveland, OH 45140','Cincinnati, OH 45242') 
final_df.loc[(final_df.address.str.contains('Grove'))&(final_df.city.str.contains('Norwood')),'formatted_address'] = final_df[(final_df.address.str.contains('Grove'))&(final_df.city.str.contains('Norwood'))]['formatted_address'].str.replace('Norwood, OH 45212','Cincinnati, OH 45227') 
final_df.loc[final_df.address.str.contains('Jefferson'),'formatted_address'] = final_df.loc[final_df.address.str.contains('Jefferson')]['formatted_address'].str.replace('Loveland, OH 45140','Cincinnati, OH 45242') 
final_df.loc[final_df.address.str.contains('Floral'),'formatted_address'] = final_df.loc[final_df.address.str.contains('Floral')]['formatted_address'].str.replace('Norwood, OH 45212','Cincinnati, OH 45242') 
final_df.loc[final_df.address.str.contains('Floral'),'formatted_address'] = final_df.loc[final_df.address.str.contains('Floral')]['formatted_address'].str.replace('Norwood, OH 45212','Cincinnati, OH 45242') 
final_df.loc[final_df.address.str.contains('Deerfield'),'formatted_address'] = final_df.loc[final_df.address.str.contains('Deerfield')]['formatted_address'].str.replace('Loveland, OH 45140','Blue Ash, OH 45242') 
final_df.loc[final_df.address.str.contains('#3 West St'),'formatted_address'] = '3759 West St, Cincinnati, OH 45227, USA'
final_df.loc[final_df.address.str.contains('805 Park'),'formatted_address'] = '805 Park Ave, Terrace Park, OH 45174'
final_df.loc[final_df.address.str.contains('Laurel'),'formatted_address'] =final_df.loc[final_df.address.str.contains('Laurel')]['formatted_address'].str.replace('Madiera, OH 45243','Cincinnati, OH 45242') 
final_df.loc[final_df.address.str.contains('35 Euclid'),'formatted_address'] = '35 Euclid Ave, Cincinnati, OH 45215, USA'
final_df.loc[final_df.address.str.contains('510 Navaho'),'formatted_address'] = '510 Navaho Dr, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('511 Navaho'),'formatted_address'] = '511 Navaho Dr, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('408 Navaho'),'formatted_address'] = '408 Navaho Dr, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('206 Navaho'),'formatted_address'] = '206 Navaho Dr, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('4 Iroquois'),'formatted_address'] = '4 Iroquois Dr, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('6757 Maple'),'formatted_address'] = '6757 Maple St, Cincinnati, OH 45227, USA'
final_df.loc[final_df.address.str.contains('4843 Laurel'),'formatted_address'] = '4843 Laurel Ave, Cincinnati, OH 45242, USA'
final_df.loc[final_df.address.str.contains('4926 Laurel'),'formatted_address'] = '4926 Laurel Ave, Cincinnati, OH 45242, USA'
final_df.loc[final_df.address.str.contains('9221 Grandview'),'formatted_address'] = '9221 Grandview Ave, Cincinnati, OH 45242, USA'
final_df.loc[final_df.address.str.contains('720 W Main'),'formatted_address'] = '720 W Main St, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('789 Brown'),'formatted_address'] = '789 Brown St, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('1301 W Main'),'formatted_address'] = '1301 W Main St, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('1401 W Main'),'formatted_address'] = '1401 W Main St, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('73 Evergreen'),'formatted_address'] = '73 Evergreen Cir, Cincinnati, OH 45215, USA'
final_df.loc[final_df.address.str.contains('5037 Bristol'),'formatted_address'] = '5037 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('5028 240 Bristol'),'formatted_address'] = '5028 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('5022 207 Bristol'),'formatted_address'] = '5022 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('5032 Bristol'),'formatted_address'] = '5032 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('410 Washington Ave'),'formatted_address'] = '410 Washington Ave, Terrace Park, OH 45174, USA'
final_df.loc[final_df.address.str.contains('5041 Bristol'),'formatted_address'] = '5041 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('5016 204 Bristol'),'formatted_address'] = '5016 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('5059 226 Bristol'),'formatted_address'] = '5059 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('411 Washington Ave'),'formatted_address'] = '411 Washington Ave, Terrace Park, OH 45174, USA'
final_df.loc[final_df.address.str.contains('418 Washington Ave'),'formatted_address'] = '418 Washington Ave, Terrace Park, OH 45174, USA'
final_df.loc[final_df.address.str.contains('5081 Bristol'),'formatted_address'] = '5081 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('5035 214 Bristol'),'formatted_address'] = '5035 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('5057 Bristol'),'formatted_address'] = '5057 Bristol Ct, Loveland, OH 45140, USA'
final_df.loc[final_df.address.str.contains('9964 Washington Ave'),'formatted_address'] = '9964 Washington Ave, Loveland, OH 45140, USA'



In [400]:
final_df.loc[final_df.address.str.contains('5028 240 Bristol')]

,parcel_number,address,finsqft,use,year_built,transfer_date,amount,total_rooms,bedrooms,full_baths,...,street,new_address,formatted_address,longitude,latitude,num_amount,year,month,missing_house_number,city
256,621-0014-0177-00,5028 240 Bristol Ct,859,550,1994,04-09-2012,"$115,000",4,1,2,...,Bristol Ct,5028 Bristol Ct 240,"5028 Bristol Ct, Loveland, OH 45140, USA",-84.303547,39.272916,115000,2012,4,5028,NaN


In [401]:
def get_address_details(address):
    response = gmaps.geocode(address)
    formatted_address = response[0]['formatted_address']
    location = response[0]['geometry']['location']
    return {
        'formatted_address': formatted_address,
        'longitude': location['lng'],
        'latitude': location['lat']
    } 

In [402]:
wrong_geocode = [
 '6977 Grace Ave'
,'4847 Myrtle Ave'
,'4912 Myrtle Ave'
,'5813 Grace Ave'
,'4200 Grove Ave'
,'4841 Myrtle Ave'
,'4850 Myrtle Ave'
,'8914 Cherry St'
,'9712 Jefferson Ave'
,'8914 Cherry St'
,'10889 Deerfield Rd'
,'9385 Floral Ave'
,'4200 Grove Ave'
,'6988 Grace Ave'
,'3759 #3 West St'
,'805 Park Ave'
,'8913 Cherry St'
,'4843 Laurel Ave'
,'8999 Cherry St'
,'35 Euclid Ave'
,'206 Navaho Dr'
,'408 Navaho Dr'
,'510 Navaho Dr'
,'511 Navaho Dr'
,'4 Iroquois Dr'
,'6757 Maple St'
,'4843 Laurel Ave'
,'4926 Laurel Ave'
,'9221 Grandview Ave'
,'720 W Main St'
,'789 Brown St'
,'1301 W Main St'
,'1401 W Main St'
,'73 Evergreen Cr'
,'5037 Bristol Ct'
,'5022 207 Bristol Ct'
,'410 Washington Ave'
,'5041 Bristol Ct'
,'5032 Bristol Ct'
,'5016 204 Bristol Ct'
,'411 Washington Ave'
,'418 Washington Ave'
,'5081 Bristol Ct'
,'5035 214 Bristol Ct'
,'9964 Washington Ave'
,'5028 240 Bristol Ct'
,'5059 226 Bristol Ct'
,'5057 Bristol Ct'
]

In [403]:
len(wrong_geocode)

48

In [404]:
final_df[final_df.address.isin(wrong_geocode)]

,parcel_number,address,finsqft,use,year_built,transfer_date,amount,total_rooms,bedrooms,full_baths,...,street,new_address,formatted_address,longitude,latitude,num_amount,year,month,missing_house_number,city
26,527-0030-0129-00,6977 Grace Ave,1427,510,1945,06-15-2009,"$158,000",7,4,2,...,Grace Ave,6977 Grace Ave,"6977 Grace Ave, Cincinnati, OH 45227, USA",-84.372584,39.149170,158000,2009,6,6977,NaN
48,621-0003-0003-00,789 Brown St,1429,510,1924,04-24-2009,"$123,000",6,2,2,...,Brown St,789 Brown St,"789 Brown St, Loveland, OH 45140, USA",-84.270244,39.269510,123000,2009,4,789,NaN
49,621-0006-0015-00,720 W Main St,1314,510,1993,06-09-2009,"$118,000",7,3,2,...,W Main St,720 W Main St,"720 W Main St, Loveland, OH 45140, USA",-84.270533,39.268196,118000,2009,6,720,NaN
51,621-0007-0049-00,4 Iroquois Dr,1260,510,1958,05-14-2009,"$110,000",5,3,1,...,Iroquois Dr,4 Iroquois Dr,"4 Iroquois Dr, Loveland, OH 45140, USA",-84.277813,39.264099,110000,2009,5,4,NaN
109,612-0040-0213-00,4847 Myrtle Ave,1357,510,1938,05-10-2010,"$159,000",7,3,1,...,Myrtle Ave,4847 Myrtle Ave,"4847 Myrtle Ave, Blue Ash, OH 45242, USA",-84.374822,39.231764,159000,2010,5,4847,NaN
131,621-0007-0247-00,510 Navaho Dr,1168,510,1999,04-29-2010,"$108,000",5,3,2,...,Navaho Dr,510 Navaho Dr,"510 Navaho Dr, Loveland, OH 45140, USA",-84.275016,39.264973,108000,2010,4,510,NaN
132,621-0007-0247-00,510 Navaho Dr,1168,510,1999,05-14-2010,"$138,000",5,3,2,...,Navaho Dr,510 Navaho Dr,"510 Navaho Dr, Loveland, OH 45140, USA",-84.275016,39.264973,138000,2010,5,510,NaN
138,621-0015-0110-00,5037 Bristol Ct,836,550,1990,04-23-2010,"$118,000",4,1,2,...,Bristol Ct,5037 Bristol Ct,"5037 Bristol Ct, Loveland, OH 45140, USA",-84.304450,39.272372,118000,2010,4,5037,NaN
139,523-0005-0082-00,5813 Grace Ave,1357,510,1930,05-26-2011,"$120,000",5,3,2,...,Grace Ave,5813 Grace Ave,"5813 Grace Ave, Cincinnati, OH 45227, USA",-84.391964,39.142769,120000,2011,5,5813,NaN
170,612-0040-0185-00,4912 Myrtle Ave,1474,510,1937,05-20-2011,"$147,000",8,3,2,...,Myrtle Ave,4912 Myrtle Ave,"4912 Myrtle Ave, Cincinnati, OH 45242, USA",-84.373898,39.232003,147000,2011,5,4912,NaN


In [405]:
results_df = final_df[final_df.address.isin(wrong_geocode)]['formatted_address'].apply(get_address_details).apply(pd.Series)

In [406]:
len(results_df)

53

In [407]:
final_df.loc[final_df.address.isin(wrong_geocode),'formatted_address'] = results_df['formatted_address'].copy()
final_df.loc[final_df.address.isin(wrong_geocode),'longitude'] = results_df['longitude'].copy()
final_df.loc[final_df.address.isin(wrong_geocode),'latitude'] = results_df['latitude'].copy()

In [409]:
final_df.to_csv('finalsold2009_2023.csv')